In [1]:
from GAME.bin.intertask_mappings import *

In [12]:
MC2D_states = ['x_position', 'x_velocity']
MC3D_states = ['x_position', 'y_position', 'x_velocity', 'y_velocity']
MC2D_actions = ['Left', 'Neutral', 'Right']
MC3D_actions = ['Neutral', 'West', 'East', 'South', 'North']

# evolution parameters
src_state_var = MC2D_states
src_actions = MC2D_actions
target_state_var = MC3D_states
target_actions = MC3D_actions

mapping = IntertaskMapping([0, 0, 0, 0], [2, 0, 2, 2, 2], src_state_var, src_actions, target_state_var, target_actions)

src_data_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\output\\10242022 Initial Samples Collection for 2D MC\\test.csv"
# src_data_df = pd.read_csv(src_data_path, index_col = False)
transformed_df_col_names = ['Current_x_position', 'Current_x_velocity', 'Current_y_position', "Current_y_velocity",
'Current_action', 'Next_x_position', 'Next_x_velocity', 'Next_y_position', 'Next_y_velocity']

# transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_actions)
transformed_df_out_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\output\\11022022 Transformed Source Data 3DMC\\transformed_df.csv"
# transformed_df.to_csv(transformed_df_out_path, index = False)
transformed_df = pd.read_csv(transformed_df_out_path, index_col = False)

network_folder_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\pickle\\11012022 3DMC Neural Nets\\"
eval_networks = EvaluationNetworks(network_folder_path)

In [16]:
MC2D_states = ['x_position', 'x_velocity']
MC3D_states = ['x_position', 'y_position', 'x_velocity', 'y_velocity']
MC2D_actions = ['Left', 'Neutral', 'Right']
MC3D_actions = ['Neutral', 'West', 'East', 'South', 'North']

# evolution parameters
src_state_var = MC2D_states
src_actions = MC2D_actions
target_state_var = MC3D_states
target_actions = MC3D_actions

mapping = IntertaskMapping([0, 0, 0, 0], [2, 0, 2, 2, 2], src_state_var, src_actions, target_state_var, target_actions)

src_data_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\output\\10242022 Initial Samples Collection for 2D MC\\test.csv"
src_data_df = pd.read_csv(src_data_path, index_col = False)
transformed_df_col_names = ['Current_x_position', 'Current_x_velocity', 'Current_y_position', "Current_y_velocity",
'Current_action', 'Next_x_position', 'Next_x_velocity', 'Next_y_position', 'Next_y_velocity']

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_actions)
# transformed_df_out_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\output\\11022022 Transformed Source Data 3DMC\\transformed_df.csv"
# transformed_df.to_csv(transformed_df_out_path, index = False)

network_folder_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\pickle\\11012022 3DMC Neural Nets\\"
eval_networks = EvaluationNetworks(network_folder_path)

In [17]:
transformed_df.head()

,Current_x_position,Current_x_velocity,Current_y_position,Current_y_velocity,Next_x_position,Next_x_velocity,Next_y_position,Next_y_velocity,Current_action_0,Current_action_1,Current_action_2,Current_action_3,Current_action_4
0,-0.445209,-0.445209,-0.445209,-0.445209,-0.444791,-0.444791,-0.444791,-0.444791,1.0,0.0,1.0,1.0,1.0
1,-0.444791,-0.444791,-0.444791,-0.444791,-0.445959,-0.445959,-0.445959,-0.445959,0.0,1.0,0.0,0.0,0.0
2,-0.445959,-0.445959,-0.445959,-0.445959,-0.447705,-0.447705,-0.447705,-0.447705,0.0,0.0,0.0,0.0,0.0
3,-0.447705,-0.447705,-0.447705,-0.447705,-0.450014,-0.450014,-0.450014,-0.450014,0.0,0.0,0.0,0.0,0.0
4,-0.450014,-0.450014,-0.450014,-0.450014,-0.452871,-0.452871,-0.452871,-0.452871,0.0,0.0,0.0,0.0,0.0


In [18]:
current_state_cols = ['Current_x_position', 'Current_x_velocity', 'Current_y_position', 'Current_y_velocity']
next_state_cols = ['Next_x_position', 'Next_x_velocity', 'Next_y_position', 'Next_y_velocity']
actions = [0, 1, 2, 3, 4]
eval_scores = {}

for action in actions:
    action_col_name = 'Current_action_' + str(action)
    src_df_by_action = transformed_df[transformed_df[action_col_name] == 1]

    features = src_df_by_action[current_state_cols]
    for target_name in next_state_cols:
        target = src_df_by_action[target_name]
        eval_mlp = eval_networks.get_network(action, target_name)
        eval_score = eval_mlp.score(features, target)
        eval_scores['{}_{}'.format(action, target_name)] = eval_score

In [19]:
eval_scores

{'0_Next_x_position': -0.17179367679357815,
 '0_Next_x_velocity': 0.9744412381716231,
 '0_Next_y_position': -0.8066205488129745,
 '0_Next_y_velocity': 0.9703719821636819,
 '1_Next_x_position': -1.7407601864417765,
 '1_Next_x_velocity': 0.377269330340398,
 '1_Next_y_position': -1.1116317726676255,
 '1_Next_y_velocity': 0.9455552985475358,
 '2_Next_x_position': -0.6225829199426247,
 '2_Next_x_velocity': 0.7096340954453446,
 '2_Next_y_position': -0.8537383263838068,
 '2_Next_y_velocity': 0.9760146478362283,
 '3_Next_x_position': -1.2309092163157414,
 '3_Next_x_velocity': 0.7078421263235803,
 '3_Next_y_position': -1.0288339514038163,
 '3_Next_y_velocity': 0.9757637613484024,
 '4_Next_x_position': -0.12452960200764873,
 '4_Next_x_velocity': 0.9743434965347499,
 '4_Next_y_position': -0.6186081398680157,
 '4_Next_y_velocity': 0.9710706463289319}

In [5]:
import os

path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\pickle\\11012022 3DMC Neural Nets\\"
for file in os.listdir(path):
    if file.endswith('.txt'):
        split_filename = file.split('_')
        new_filename = split_filename[0] + '--' + split_filename[1].split('_')[0] + '-' + split_filename[1].split('_')[1] + '_' + split_filename[1].split('_')[2]
        # os.rename(os.path.join(path, file), os.path.join(path, new_filename))
        print(new_filename)

a0--sNext-x_position.pickle
a0--sNext-x_velocity.pickle
a0--sNext-y_position.pickle
a0--sNext-y_velocity.pickle
a1--sNext-x_position.pickle
a1--sNext-x_velocity.pickle
a1--sNext-y_position.pickle
a1--sNext-y_velocity.pickle
a2--sNext-x_position.pickle
a2--sNext-x_velocity.pickle
a2--sNext-y_position.pickle
a2--sNext-y_velocity.pickle
a3--sNext-x_position.pickle
a3--sNext-x_velocity.pickle
a3--sNext-y_position.pickle
a3--sNext-y_velocity.pickle
a4--sNext-x_position.pickle
a4--sNext-x_velocity.pickle
a4--sNext-y_position.pickle
a4--sNext-y_velocity.pickle
